In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd

from collections import defaultdict
from nird.utils import load_config
import nird.road_functions as post_func
import warnings

warnings.simplefilter("ignore")
base_path = Path(load_config()["paths"]["base_path"])

## Damage Analysis

In this example, we computed the direct flood damages on GB roads based on JBA synthetic event (Thames Lloyd's RDS). Direct damage analysis involves estimating the degree of functionality lost in flood of different severity, the costs related to clean-up and repair of physcial road infrastructure assets that are exposed to flooding, and the estimated recovery time. 

### Intersection analysis

In [4]:
raster_root = Path(load_config()["paths"]["JBA_data"]) / "completed"

In [5]:
# load road links
road_links = gpd.read_parquet(base_path / "networks" / "road" / "GB_road_links_with_bridges.gpq")
road_links.head(3)

,from_id,to_id,e_id,fictitious,road_classification,road_function,form_of_way,road_classification_number,name_1,name_1_lang,...,average_toll_cost,urban,averageWidth,minimumWidth,max_z,min_z,mean_z,lanes,geometry,road_label
0,67C5F67E-3D2E-4EAE-8546-99A0B55D8094,32A6AE75-AAD5-456E-8CF4-B7012B520D4E,roade_0,False,A Road,A Road,Single Carriageway,A4017,Bromley Heath Road,None,...,0.0,0,16.1,15.4,41.7,41.6,41.650000,4,"LINESTRING (364712 178031, 364713.1 178027.1, ...",road
1,33FEA706-7956-41E3-9825-11D6831837DF,648DD533-65BD-4148-93ED-683D6D0E1C40,roade_1,False,A Road,A Road,Single Carriageway,A85,None,None,...,0.0,0,7.2,6.8,148.5,147.5,148.025000,2,"LINESTRING (250534.93 728166.94, 250596.46 728...",road
2,10DE2E65-4BED-4F49-9ED5-E4D9D872079B,C58D254E-B492-4C2A-8863-ECD569F30988,roade_2,False,A Road,A Road,Roundabout,A642,None,None,...,0.0,0,10.2,8.2,50.4,49.6,49.966667,2,"LINESTRING (435874 425820, 435857 425814, 4358...",road


In [6]:
# load baseline edge flow simulation results
base_scenario_links = gpd.read_parquet(
    base_path.parent / "outputs" / "base_scenario" / "edge_flows_visualisation.pq"
)
base_scenario_links.rename(
    columns={
        "acc_capacity": "current_capacity",
        "acc_speed": "current_speed",
        "acc_flow": "current_flow",
    },
    inplace=True,
)

In [7]:
# load flood hazard data
raster_path = raster_root / "river" / "17_Thames Lloyd's RDS"/ "Raster" / "UK_RDS_LloydsRDS_Thames_Flood_FLRF_RD_5m_4326.tif"
# load clip mask
clip_path = raster_root / "river" / "17_Thames Lloyd's RDS"/ "Vector" / "UK_RDS_LloydsRDS_Thames_Flood_FLRF_VE_5m_4326.shp"

In [8]:
# intersection analysis (using JBA sythetic flood hazard data for demonstration)
"""
- intersect road links with flood hazard data -> floodwater depth
- estimate overtopping floodwater depth (subtract floodwater depth by embankment height for riverine flood)
- estimate damage level for road segments based on overtopping floodwater depth
"""
intersections = post_func.intersections_with_damage(
    road_links,
    flood_key= "Thames_Lloyds_RDS",
    flood_type="river",
    flood_path=raster_path,
    clip_path=clip_path,
)
intersections.head(3)

Clipping features based on Thames_Lloyds_RDS...
Intersecting features with raster Thames_Lloyds_RDS...


,from_id,to_id,e_id,fictitious,road_classification,road_function,form_of_way,road_classification_number,name_1,name_1_lang,...,min_z,mean_z,lanes,road_label,geometry,split,index_i,index_j,flood_depth_river,damage_level_river
0,25694CD4-1ADE-495D-85B6-94DB1DB119F9,20CDC722-CA3B-4D39-9ED4-74984B7E0ED0,roade_511,False,A Road,A Road,Single Carriageway,A308,Staines Road East,None,...,10.9,11.170833,2,road,"LINESTRING (511274 169498, 511273.88 169497.988)",0,20551,8532,0.760126,moderate
1,25694CD4-1ADE-495D-85B6-94DB1DB119F9,20CDC722-CA3B-4D39-9ED4-74984B7E0ED0,roade_511,False,A Road,A Road,Single Carriageway,A308,Staines Road East,None,...,10.9,11.170833,2,road,"LINESTRING (511273.88 169497.988, 511273.244 1...",1,20550,8532,0.755666,moderate
2,25694CD4-1ADE-495D-85B6-94DB1DB119F9,20CDC722-CA3B-4D39-9ED4-74984B7E0ED0,roade_511,False,A Road,A Road,Single Carriageway,A308,Staines Road East,None,...,10.9,11.170833,2,road,"LINESTRING (511273.244 169497.926, 511270.756 ...",2,20550,8533,0.862653,moderate


In [9]:
# calculate "damage level" and "maximum overtopping floodwater depth" for road links by integrating the results of their corresponding segments
road_links_with_damage = post_func.features_with_damage(road_links, intersections)

In [ ]:
# attach post-flood (D-0) capacity and speed to road links
road_links_with_damage = road_links_with_damage.merge(
    base_scenario_links[
        [
            "e_id",
            "combined_label",
            "free_flow_speeds",
            "initial_flow_speeds",
            "min_flow_speeds",
            "current_capacity",
            "current_speed",
            "current_flow",
        ]
    ],
    how="left",
    on="e_id",
)

### Road functionality loss estimate

##### Speed restrictions

In [16]:
# estimate speed restructions for inundated road links
road_links_with_damage["max_speed"] = road_links_with_damage.apply(
    lambda row: post_func.compute_maximum_speed_on_flooded_roads(
        row["flood_depth_max"],
        row["free_flow_speeds"],
        threshold=30, # floodwater deopth threshold for road closure (cm) with options [15, 30, 60]
    ),
    axis=1,
)

In [17]:
road_links_with_damage.head(3)

,from_id,to_id,e_id,fictitious,road_classification,road_function,form_of_way,road_classification_number,name_1,name_1_lang,...,flood_depth_max,damage_level_max,combined_label,free_flow_speeds,initial_flow_speeds,min_flow_speeds,current_capacity,current_speed,current_flow,max_speed
0,67C5F67E-3D2E-4EAE-8546-99A0B55D8094,32A6AE75-AAD5-456E-8CF4-B7012B520D4E,roade_0,False,A Road,A Road,Single Carriageway,A4017,Bromley Heath Road,None,...,0.0,no,A_single,37.0,37.0,0.2,117454,37.0,26546,37.0
1,33FEA706-7956-41E3-9825-11D6831837DF,648DD533-65BD-4148-93ED-683D6D0E1C40,roade_1,False,A Road,A Road,Single Carriageway,A85,None,None,...,0.0,no,A_single,37.0,37.0,0.2,71350,37.0,650,37.0
2,10DE2E65-4BED-4F49-9ED5-E4D9D872079B,C58D254E-B492-4C2A-8863-ECD569F30988,roade_2,False,A Road,A Road,Roundabout,A642,None,None,...,0.0,no,A_dual,45.0,45.0,0.2,72000,45.0,0,45.0


##### Traffic restrictions
 Structural damages along do not necessarily cause traffic disruptions as traffic loads only represent a portion of the total design capacity of roads. 

 Therefore, we assumed that roads with minor to moderate damages (i.e., damage_level_max) would remain operational but with reduced capacities (i.e., current capacity) and traffic speeds (i.e., current speed) adjusted based on traffic flow conditions at the time of the flood event. 

 While roads with extensive and severe damages (i.e., damage_level_max) were considered inoperable during repair and reconstruction, halting traffic flow entirely. 

##### Road recovery time estimate
After the estimation of road functionality lost in floods of different severity, the set of failed road assets and bridges were identified. We introduced a novel recovery model to estimate recovery duration for fixing damaged roads and bridges. 
- A maxumum recovery time of 110 days was assumed after the occurence of flood event. 
- For bridges, we assumed a certain idle time of 9, 20, 35, and 50 days for those with minor, moderate, extensive, and severe damages; during teh recovery time, we assumed the capacity would be restored at linear rate of recovery, leading to a daily restoration rate of 17%, 4%, 3% and 2%. 
- For non-bridge road assets, we assumed the capacity of most would be fulled recovered within 24 hours, except for severely damaged ones of which the capacity would restore within next two days.  

### Direct damage costs estimate

In [28]:
# load damage/vulnerability curves
damages_ratio_df = pd.read_excel(
    base_path / "tables" / "damage_ratio_road_flood.xlsx"
)
damage_curves = post_func.create_damage_curves(damages_ratio_df)

In [29]:
# load damage cost values
road_links = gpd.read_parquet(
    base_path / "networks" / "road" / "GB_road_links_with_bridges.gpq"
)
road_damage_file = pd.read_excel(
    base_path / "tables" / "damage_cost_road_flood_uk.xlsx", sheet_name="roads"
)
tunnel_damage_file = pd.read_excel(
    base_path / "tables" / "damage_cost_road_flood_uk.xlsx",
    sheet_name="tunnels",
)
bridge_surface_damage_file = pd.read_excel(
    base_path / "tables" / "damage_cost_road_flood_uk.xlsx",
    sheet_name="bridges-surface",
)
bridge_river_damage_file = pd.read_excel(
    base_path / "tables" / "damage_cost_road_flood_uk.xlsx",
    sheet_name="bridges-river",
)
dv_road_dict = defaultdict(lambda: defaultdict(float))
for row in road_damage_file.itertuples():
    dv_road_dict[row.label]["min"] = row.Min
    dv_road_dict[row.label]["max"] = row.Max
    dv_road_dict[row.label]["mean"] = row.Mean

dv_tunnel_dict = defaultdict(lambda: defaultdict(float))
for row in tunnel_damage_file.itertuples():
    dv_tunnel_dict[row.label]["min"] = row.Min
    dv_tunnel_dict[row.label]["max"] = row.Max
    dv_tunnel_dict[row.label]["mean"] = row.Mean

dv_bridge_surface_dict = defaultdict(lambda: defaultdict(float))
for row in bridge_surface_damage_file.itertuples():
    dv_bridge_surface_dict[row.label]["min"] = row.min
    dv_bridge_surface_dict[row.label]["max"] = row.max
    dv_bridge_surface_dict[row.label]["mean"] = row.mean

dv_bridge_river_dict = defaultdict(lambda: defaultdict(float))
for row in bridge_river_damage_file.itertuples():
    dv_bridge_river_dict[row.label]["min"] = row.min
    dv_bridge_river_dict[row.label]["max"] = row.max
    dv_bridge_river_dict[row.label]["mean"] = row.mean

damage_values = {
    "road": dv_road_dict,
    "tunnel": dv_tunnel_dict,
    "bridge_surface": dv_bridge_surface_dict,
    "bridge_river": dv_bridge_river_dict,
}

In [53]:
# load intersections
intersections = pd.read_parquet(base_path.parent / "outputs" / "disruption_analysis" / "20241229"/ "30" / "intersections" /"intersections_17.pq")

In [54]:
# format intersections
intersections = post_func.format_intersections(intersections, road_links)


In [55]:
# run damage analysis for intersections
""" To compute:
- damage fractions based on overtopping floodwater depth and different damage curves
- damage values basedon damage fractions and different damage cost values
"""
intersections_with_damage = post_func.calculate_damage(intersections, damage_curves, damage_values)

In [69]:
# aggregate damage values for road links
# attributes
intersections_with_damage_gp1 = (
    intersections_with_damage[
        [
            "e_id",
            "road_classification",
            "form_of_way",
            "trunk_road",
            "urban",
            "lanes",
            "averageWidth",
            "road_label",
        ]
    ]
    .groupby(by=["e_id"], as_index=False)
    .first()
)
# numeric values
intersections_with_damage_gp2 = (
    pd.concat(
        [
            intersections_with_damage["e_id"],
            intersections_with_damage["length"],
            intersections_with_damage.iloc[:, -52:],
        ],
        axis=1,
    )
    .fillna(0)
    .groupby(by=["e_id"], as_index=False)
    .sum()  # sum up damage values of all segments -> disrupted links
)
# merge attributes and numeric values
common_cols = intersections_with_damage_gp1.columns.intersection(intersections_with_damage_gp2.columns).tolist()
intersections_with_damage_final = intersections_with_damage_gp1.merge(
    intersections_with_damage_gp2, on=common_cols, how="left"
)
# modify the value of damage fractions to be no greater than 1.0
fraction_cols = intersections_with_damage_final.filter(like="fraction").columns
intersections_with_damage_final[fraction_cols] = (
    intersections_with_damage_final[fraction_cols].clip(upper=1.0)
)

In [70]:
intersections_with_damage_final

,e_id,road_classification,form_of_way,trunk_road,urban,lanes,averageWidth,road_label,length,C1_surface_damage_fraction,...,C5_surface_damage_value_mean,C5_river_damage_value_min,C5_river_damage_value_max,C5_river_damage_value_mean,C6_surface_damage_value_min,C6_surface_damage_value_max,C6_surface_damage_value_mean,C6_river_damage_value_min,C6_river_damage_value_max,C6_river_damage_value_mean
0,roade_100968,A Road,Single Carriageway,False,0,2,9.520000,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,roade_101803,A Road,Collapsed Dual Carriageway,False,0,4,17.600000,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,roade_101930,A Road,Single Carriageway,False,0,2,8.918182,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,roade_101931,A Road,Single Carriageway,False,0,2,8.918182,bridge,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,roade_101932,A Road,Single Carriageway,False,0,2,8.918182,bridge,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1649,roade_98378,A Road,Single Carriageway,False,0,2,6.325000,bridge,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1650,roade_98379,A Road,Single Carriageway,False,0,2,6.325000,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1651,roade_98692,A Road,Collapsed Dual Carriageway,False,0,6,24.350000,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1652,roade_98714,A Road,Slip Road,False,0,2,9.950000,road,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
